In [1]:
import pandas as pd
import requests
import re
from collections import Counter
from typing import List, Dict

from consts import *

In [2]:
df = pd.read_csv('kns_csv_files/kns_committee.csv')
df = df[df['KnessetNum'] >= 25]
df = df[df['CategoryID'].isin([MONEY_COM_CATEGORY_ID, DEFENSE_COM_CATEGORY_ID, LAW_ORDER_COM_CATEGORY_ID, MESADERET_COM_CATEGORY_ID, KNESSET_COM_CATEGORY_ID])]
commitee_ids = df['CommitteeID'].to_list()

In [3]:
def get_meeting_protocol_text(text_path):
    # Define the URL to fetch
    base_url = 'https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/'
    # Send GET request to the URL
    response = requests.get(base_url + text_path)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        # Retrieve the content of the file
        return response.text
    else:
        raise ValueError(f"Failed to retrieve content. Status code: {response.status_code}")


In [4]:
com_session_df = pd.read_csv('kns_csv_files/kns_committeesession.csv')
com_session_df = com_session_df[com_session_df['CommitteeID'].isin(commitee_ids)]

com_session_df.dropna(subset=['text_parsed_filename'], inplace=True)
text_paths = com_session_df['text_parsed_filename'].to_list()
texts = [get_meeting_protocol_text(path) for path in text_paths]


In [5]:
knesset_members_df = pd.read_csv('kns_csv_files/kns_person.csv')
first_names, last_names = knesset_members_df['FirstName'].to_list(), knesset_members_df['LastName'].to_list()
knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

warnings = dict.fromkeys(knesset_members, [0, 0, 0])

# handle members with a middle name or a nickname
new_first_names, new_last_names = [], []

for fn, ln in zip(first_names, last_names):
    names = re.findall('\w+', fn)
    
    for name in names:
        warnings[name + ' ' + ln] = warnings[fn + ' ' + ln]
        new_first_names.append(name)
        new_last_names.append(ln)

# update first and last names
first_names += new_first_names
last_names += new_last_names

knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

In [8]:
def get_meeting_warnings(text, warnings, knesset_members) -> None:
    """
    Return warnings from the meeting protocol text.

    Parameters
    ----------
    text : str
        Meeting protocol text.

    warnings: Dict[str, List[int]]
        Number of warnings for each Knesset member.

    knesset_members: List[str]
        List of Knesset members.
    """

    # find all warnings
    matches = re.findall(rex, text, flags=re.MULTILINE)
    print(matches)
    print(len(matches))
    for m in matches:
        sentences = m.split('\n')
        first_sentence, last_sentence = sentences[0], sentences[-1]
        for kns_member in knesset_members:
            if kns_member in first_sentence:
                word2idx = {'ראש': 0, 'שני': 1, 'שליש': 2}
                for word, idx in word2idx.items():
                    if word in last_sentence:
                        warnings[kns_member][idx] += 1
                        break
    
    

In [9]:
with open('protocols/2200827.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    print(text)

get_meeting_warnings(text, warnings, knesset_members)

פרוטוקול של ישיבת ועדה

הכנסת העשרים-וחמש

הכנסת



3
ועדת החוקה, חוק ומשפט
06/02/2023


מושב ראשון



פרוטוקול מס' 27
מישיבת ועדת החוקה, חוק ומשפט
יום שני, ט"ו בשבט התשפ"ג (06 בפברואר 2023), שעה 9:30



סדר היום:
 << נושא >> צִיּוֹן בְּמִשְׁפָּט תִּפָּדֶה - מחזירים את הצדק למערכת המשפט, נושא הישיבה: הצעת חוק מטעם הוועדה - הצעת חוק-יסוד: השפיטה (חיזוק הפרדת הרשויות) << נושא >>   


נכחו:
חברי הוועדה: 
שמחה רוטמן – היו"ר
חנוך דב מלביצקי – מ"מ היו"ר
אריאל קלנר – מ"מ היו"ר
קארין אלהרר
אברהם בצלאל
טלי גוטליב
עופר כסיף
יואב סגלוביץ'
משה סעדה
יצחק פינדרוס
אורית פרקש הכהן
גלעד קריב


חברי הכנסת:
דבי ביטון
רם בן ברק
יוסף טייב
רון כץ
יוראי להב הרצנו
ארז מלול
שורן ניר
גדעון סער
מנסור עבאס
עידן רול
אפרת רייטן מרום
נאור שירי
אלעזר שטרן


מוזמנים: 
	אביטל סומפולינסקי
	–
	המשנה ליועצת המשפטית לממשלה (משפט ציבורי - חוקתי), משרד המשפטים

	גאל אזריאל
	–
	מחלקת יעוץ וחקיקה, משרד המשפטים

	נורית קורן 
	–
	חברת כנסת לשעבר

	מוסי רז
	–
	חבר כנסת לשעבר

	איתן גינצבורג
	–
	חבר כנסת לשעבר

	יהודה בן מאיר
	–
	

In [10]:
warnings

{'בנימין אלון': [36, 32, 8],
 'צבי הנדל': [36, 32, 8],
 'רחבעם זאבי': [36, 32, 8],
 'אליעזר כהן': [36, 32, 8],
 'אביגדור ליברמן': [36, 32, 8],
 'יורי שטרן': [36, 32, 8],
 'קולט אביטל': [36, 32, 8],
 'אפי אושעיה': [36, 32, 8],
 'דליה איציק': [36, 32, 8],
 'אברהם בורג': [36, 32, 8],
 'בנימין בן-אליעזר': [36, 32, 8],
 'אלי בן-מנחם': [36, 32, 8],
 'שלמה בן-עמי': [36, 32, 8],
 'יעל דיין': [36, 32, 8],
 'סאלח טריף': [36, 32, 8],
 'אברהם יחזקאל': [36, 32, 8],
 'איתן כבל': [36, 32, 8],
 'רענן כהן': [36, 32, 8],
 'יוסי כץ': [36, 32, 8],
 'סופה לנדבר': [36, 32, 8],
 'מיכאל מלכיאור': [36, 32, 8],
 'נואף מסאלחה': [36, 32, 8],
 'אפרים סנה': [36, 32, 8],
 'אופיר פינס-פז': [36, 32, 8],
 'שמעון פרס': [36, 32, 8],
 'חיים רמון': [36, 32, 8],
 'אברהם-בייגה שוחט': [36, 32, 8],
 'ויצמן שירי': [36, 32, 8],
 'שלום שמחון': [36, 32, 8],
 'מיכאל איתן': [36, 32, 8],
 'משה ארנס': [36, 32, 8],
 'זאב בוים': [36, 32, 8],
 'נעמי בלומנטל': [36, 32, 8],
 'אברהם הירשזון': [36, 32, 8],
 'צחי הנגבי': [36, 32, 8],
 'ישראל 